# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Import necessary packages

In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales_data = pd.read_csv('kc_house_data.csv',
                         dtype = {'bathrooms':float, 'waterfront':int, 
                                  'sqft_above':int, 'sqft_living15':float, 
                                  'grade':int, 'yr_renovated':int, 
                                  'price':float, 'bedrooms':float, 
                                  'zipcode':str, 'long':float, 
                                  'sqft_lot15':float, 'sqft_living':float, 
                                  'floors':str, 'condition':int, 'lat':float, 
                                  'date':str, 'sqft_basement':int, 
                                  'yr_built':int, 'id':str, 'sqft_lot':int,
                                  'view':int})
sales = pd.DataFrame(sales_data)

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [31]:
train_data = pd.read_csv('kc_house_train_data.csv',
                         dtype = {'bathrooms':float, 'waterfront':int, 
                                  'sqft_above':int, 'sqft_living15':float, 
                                  'grade':int, 'yr_renovated':int, 
                                  'price':float, 'bedrooms':float, 
                                  'zipcode':str, 'long':float, 
                                  'sqft_lot15':float, 'sqft_living':float, 
                                  'floors':str, 'condition':int, 'lat':float, 
                                  'date':str, 'sqft_basement':int, 
                                  'yr_built':int, 'id':str, 'sqft_lot':int,
                                  'view':int})

test_data = pd.read_csv('kc_house_test_data.csv',
                         dtype = {'bathrooms':float, 'waterfront':int, 
                                  'sqft_above':int, 'sqft_living15':float, 
                                  'grade':int, 'yr_renovated':int, 
                                  'price':float, 'bedrooms':float, 
                                  'zipcode':str, 'long':float, 
                                  'sqft_lot15':float, 'sqft_living':float, 
                                  'floors':str, 'condition':int, 'lat':float, 
                                  'date':str, 'sqft_basement':int, 
                                  'yr_built':int, 'id':str, 'sqft_lot':int,
                                  'view':int})

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [33]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = LinearRegression()
example_model.fit(train_data[example_features], train_data['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [34]:
example_weight_summary = np.insert(example_model.coef_, 0, example_model.intercept_)
print (example_weight_summary)

[ 87912.86581496    315.40669062 -65081.88711588   6942.16598637]


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [27]:
example_predictions = example_model.predict(train_data[example_features])
print (example_predictions[0]) # should be 271789.505878

271789.26537996985


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [20]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    errors = outcome - predictions
    # Then square and add them up
    square_errors = errors * errors
    RSS = square_errors.sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [35]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data[example_features], test_data['price'])
print (rss_example_train) # should be 2.7376153833e+14

273761940583133.72


In [22]:
rss_example_train/10**14

2.737619405831337

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [24]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [37]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [40]:
# create the remaining 3 features in both TEST and TRAIN data
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']

bedrooms_squared: 12.4466777015843
bed_bath_rooms: 7.5039016315913925
log_sqft_living: 7.550274679645921
lat_plus_long: -74.65333355403185
bedrooms_squared: 12.174
bed_bath_rooms: 7.4966
log_sqft_living: 7.5503
lat_plus_long: -74.654


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

In [41]:
train_data.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,0,98178,47.5112,-122.257,1340.0,5650.0,9.0,3.00,7.073270,-74.7458
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,1991,98125,47.7210,-122.319,1690.0,7639.0,9.0,6.75,7.851661,-74.5980
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,0,98028,47.7379,-122.233,2720.0,8062.0,4.0,2.00,6.646391,-74.4951


**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [43]:
#print mean values of 4 new features on TEST dataTest data
print ("bedrooms_squared: {:.3}".format(test_data['bedrooms_squared'].mean()))
print ("bed_bath_rooms: {:.3}".format(test_data['bed_bath_rooms'].mean()))
print ("log_sqft_living: {:.3}".format(test_data['log_sqft_living'].mean()))
print ("lat_plus_long: {:.3}".format(test_data['lat_plus_long'].mean()))
#print mean values of 4 new features on train dataTest data
print ("bedrooms_squared: {}".format(train_data['bedrooms_squared'].mean()))
print ("bed_bath_rooms: {}".format(train_data['bed_bath_rooms'].mean()))
print ("log_sqft_living: {}".format(train_data['log_sqft_living'].mean()))
print ("lat_plus_long: {}".format(train_data['lat_plus_long'].mean()))

bedrooms_squared: 12.4
bed_bath_rooms: 7.5
log_sqft_living: 7.55
lat_plus_long: -74.7
bedrooms_squared: 12.174240681086056
bed_bath_rooms: 7.496591693511275
log_sqft_living: 7.550349317684997
lat_plus_long: -74.65396803382448


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [44]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [45]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = LinearRegression()
model_1.fit(train_data[model_1_features], train_data['price'])
model_2 = LinearRegression()
model_2.fit(train_data[model_2_features], train_data['price'])
model_3 = LinearRegression()
model_3.fit(train_data[model_3_features], train_data['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [46]:
# Examine/extract each model's coefficients:
model_1_weight_summary = np.insert(model_1.coef_, 0, model_1.intercept_)
print (model_1_weight_summary)
model_2_weight_summary = np.insert(model_2.coef_, 0, model_2.intercept_)
print (model_2_weight_summary)
model_3_weight_summary = np.insert(model_3.coef_, 0, model_3.intercept_)
print (model_3_weight_summary)

[-6.90757268e+07  3.12258646e+02 -5.95865332e+04  1.57067421e+04
  6.58619264e+05 -3.09374351e+05]
[-6.68679689e+07  3.06610053e+02 -1.13446368e+05 -7.14613083e+04
  6.54844630e+05 -2.94298969e+05  2.55796520e+04]
[-6.20360850e+07  5.29422820e+02  3.45142296e+04  6.70607813e+04
  5.34085611e+05 -4.06750711e+05 -8.57050439e+03 -6.78858667e+03
 -5.61831484e+05  1.27334900e+05]


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [49]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_1_train = get_residual_sum_of_squares(model_1,train_data[model_1_features],train_data['price'])
print ("RSS of model 1 on training data: {}".format(rss_1_train))
rss_2_train = get_residual_sum_of_squares(model_2,train_data[model_2_features],train_data['price'])
print ("RSS of model 2 on training data: {}".format(rss_2_train))
rss_3_train = get_residual_sum_of_squares(model_3,train_data[model_3_features],train_data['price'])
print ("RSS of model 3 on training data: {}".format(rss_3_train))

RSS of model 1 on training data: 967879963049545.8
RSS of model 2 on training data: 958419635074070.0
RSS of model 3 on training data: 903436455050479.2


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [52]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_1_test = get_residual_sum_of_squares(model_1,test_data[model_1_features],test_data['price'])
print ("RSS of model 1 on training data: {}".format(rss_1_test))
rss_2_test = get_residual_sum_of_squares(model_2,test_data[model_2_features],test_data['price'])
print ("RSS of model 2 on training data: {}".format(rss_2_test))
rss_3_test = get_residual_sum_of_squares(model_3,test_data[model_3_features],test_data['price'])
print ("RSS of model 3 on training data: {}".format(rss_3_test))

RSS of model 1 on training data: 225500469795490.34
RSS of model 2 on training data: 223377462976467.5
RSS of model 3 on training data: 259236319207179.94


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.